In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import glob
import math
import numpy as np
import pandas as pd

In [3]:
import trackml
from trackml.dataset import load_event, load_dataset

In [4]:
def areAlignedRZ(hi, r1, z1, ho, ptmin, thetaCut) :
    ro = ho['r']
    zo = ho['z']
    radius_diff = abs(r1 - ro)
    distance_13_squared = radius_diff*radius_diff + (z1 - zo)*(z1 - zo)
    
    pMin = ptmin*np.sqrt(distance_13_squared) #this needs to be divided by radius_diff later
    
    tan_12_13_half_mul_distance_13_squared = abs(z1 * (hi['r'] - ro) + hi['z'] * (ro - r1) + zo * (r1 - hi['r'])) 
    return tan_12_13_half_mul_distance_13_squared * pMin <= thetaCut * distance_13_squared * radius_diff

In [10]:
# file = '/Users/innocent/data/trackML_minitrain.zip'
file = '/Users/innocent/data/train_100_events'
maxEv = 2
evn=0
for event_id, hits, cells, particles, truth in load_dataset(file):
    evn+=1
    if (evn>maxEv) : break
    print event_id, len(hits)
    hits['phi'] = np.arctan2(hits['y'],hits['x'])
    hits['r'] = np.sqrt(hits['y']*hits['y']+hits['x']*hits['x'])
    hits['up0'] =0
    hits['up1'] =0
    hits.sort_values(by=['phi'],inplace=True)
    hl1 = hits.query('volume_id ==8 & layer_id==2')
    up1 = np.zeros([len(hl1),2],dtype=int)
    hl2 = hits.query('volume_id ==8 & layer_id==4')
    up2 = np.zeros([len(hl2),2],dtype=int)
    hl3 = hits.query('volume_id ==8 & layer_id==6')
    up3 = np.zeros([len(hl3),2],dtype=int)
    hl4 = hits.query('volume_id ==8 & layer_id==8')
    print len(hl1), len(hl2), len(hl3), len(hl4)
    #print hl1
    # print hl2
    #print hits.loc[lambda h: h.volume_id ==8 & h.layer_id==2]
    #print hits[hits['volume_id']==8 & hits['layer_id']==2]
    nd=0
    ih=0
    for h1 in hl1.itertuples() :
        phi = h1.phi
        hh = hl2['phi'].searchsorted([phi-0.2,phi+0.2])
        up1[ih] = hh
        hits.loc[h1.Index,'up0'] = hh[0]
        hits.loc[h1.Index,'up1'] = hh[1]
        ih+=1
        nd += hh[1]-hh[0]
#        print h1.values, hh, hl2.iloc[hh[0]],hl2.iloc[hh[1]]
#        break
    # h1 = pd.concat(h1,up1)
    print h1
    print hits.loc[h1.Index]
    hl1 = hits.query('volume_id ==8 & layer_id==2')
    print next(hl1.iterrows())
    print 'd12', nd # len(d12)
    nd=0
    ih=0
    for h2 in hl2.itertuples() :
        phi = h2.phi
        hh = hl3['phi'].searchsorted([phi-0.2,phi+0.2])
        hits.loc[h2.Index,'up0'] = hh[0]
        hits.loc[h2.Index,'up1'] = hh[1]
        up2[ih] = hh
        ih+=1
        nd += hh[1]-hh[0]
#        print h1.values, hh, hl2.iloc[hh[0]],hl2.iloc[hh[1]]
#        break
    print 'd23',nd # len(d12)
    ih=0
    for h3 in hl3.itertuples() :
        phi = h3.phi
        hh = hl4['phi'].searchsorted([phi-0.2,phi+0.2])
        hits.loc[h3.Index,'up0'] = hh[0]
        hits.loc[h3.Index,'up1'] = hh[1]
        up3[ih] = hh
        ih+=1
        nd += hh[1]-hh[0]
#        print h1.values, hh, hl2.iloc[hh[0]],hl2.iloc[hh[1]]
#        break
    print 'd34',nd # len(d12)
    
    CAThetaCut = 0.0010
    CAPhiCut = 0.175
    ptMin = 0.6 # algo tuned for 4T so need to scale up x2
    nd=0
    na=0
    i1=0
    for hi1,h1 in hl1.iterrows() :
        hh1 = up1[i1]
        i1+=1
        i2=0
        for hi2,h2 in hl2.iloc[hh1[0]:hh1[1]].iterrows() :
            if abs(h2['z']-h1['z'])>300 : continue
            hh2 = up2[i2]
            i2+=1
            szh3 = hl3.iloc[hh2[0]:hh2[1]].loc[lambda df : abs(h2['z']-df['z'])<200]
            na += len(szh3)
            sh3 = szh3.loc[lambda df : areAlignedRZ(h1,h2['r'],h2['z'],df,ptMin,CAThetaCut)]
            nd += len(sh3)
#            for hi3,h3 in hl3.iloc[hh2[0]:hh2[1]].iterrows() :
#                na+=1
#                if (areAlignedRZ(h1,h2['r'],h2['z'],h3['r'],h3['z'],ptMin,CAThetaCut)) : nd+=1
        print i1, 't123', na, nd # len(d12)
    print 't123', na, nd # len(d12)
    
print 'done', evn

1000 120939
8892 7484 6672 6124
Pandas(Index=23559, hit_id=23560, x=-33.83039855957031, y=0.0009540169849060476, z=71.31159973144531, volume_id=8, layer_id=2, module_id=129, phi=3.14156436920166, r=33.83039855957031, up0=0, up1=0)
hit_id       23560.000000
x              -33.830399
y                0.000954
z               71.311600
volume_id        8.000000
layer_id         2.000000
module_id      129.000000
phi              3.141564
r               33.830399
up0           7256.000000
up1           7484.000000
Name: 23559, dtype: float64
(21456, hit_id       21457.000000
x              -33.304798
y               -0.002312
z               70.182999
volume_id        8.000000
layer_id         2.000000
module_id      113.000000
phi             -3.141523
r               33.304798
up0              0.000000
up1            212.000000
Name: 21456, dtype: float64)
d12 4194833
d23 3148475


KeyboardInterrupt: 